<a href="https://colab.research.google.com/github/Hugo-Zh0/YoloV12-Object-Detection-Project/blob/main/YOLOv12_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🚀 YOLOv12 Object Detection Project

[![Python](https://img.shields.io/badge/Python-3.10%2B-blue.svg)](https://www.python.org/downloads/)
[![Anaconda](https://img.shields.io/badge/Anaconda-Navigator-green.svg)](https://www.anaconda.com/download)
[![VS Code](https://img.shields.io/badge/Editor-VS%20Code-blue.svg)](https://code.visualstudio.com/)
[![Ultralytics](https://img.shields.io/badge/YOLOv12-Ultralytics-yellow.svg)](https://github.com/ultralytics/ultralytics)
[![License](https://img.shields.io/badge/License-MIT-lightgrey.svg)](LICENSE)

---
<br>

## 📌 Overview
A **collaborative group project** by Swinburne University students in partnership with **CSIRO**.  
This repository contains the setup, configuration, and workflow for training and running **YOLOv12** object detection models.

**👨‍💻 Team Members:** Harron, Feng, Bunmi, Huss, Hugo.

---
**Repo:** `Hugo-Zh0/YoloV12-Object-Detection-Project`  
**What you’ll do:**
1. Check runtime & GPU
2. Install dependencies
3. Clone your repo
4. Set paths in repository
5. Train
6. Validate
7. Predict
8. Export
10. Troubleshoot

<br>

#### **Full Process Run Through**

Process 1: Train > Validate > Inference > (repeat steps) to get multiple models with different results

Processs 2: Run Final Model > Gets Final Model > Run Full Test

Process 3: Final Model > Export Model (for deployment)

<br>

#### **What to do after completion or if you don't want to run anymore**

After completing this colab you will need to export the folders(step 9) which includes the repository and runs, as the runtime session will expire when you close the website **(meaning the folders gets deleted)**.

<br>

#### **Starting from previous session**

If you are rerunning this agin, you will need to manually upload the folders back into the google colab again (it has to be zipped up first to be uploaded)
Then you will need to run script to extract the folders back to original state.

From there you can start from Step 1,2,5,6,7,8 (excludes 3-4 as no need to clone repository and setting paths again)

## **Step 1 — 🚀 Runtime & GPU check**

## **Prerequistes**
*   Change runtime type to T4-GPU
*   Change runtime to Python 3
*   Have your dataset already downloaded







In [ ]:
#@title Check Python, CUDA, and PyTorch (Checks if runtime is all correct)

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not installed yet (will install in next step).")

## Step 2 — ⬇️ Install dependencies

These are the python libraries and Ultralytics libraries needed to run the framework

In [ ]:
#@title Install Ultralytics & helpers

!pip install ultralytics

import torch, cv2, ultralytics
print("Ultralytics:", ultralytics.__version__)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("OpenCV:", cv2.__version__)

## Step 3 —  🤖 Clone your repository

Grabs our repository which contains our folder structure/files and folders to get started with our training

**(if you already have a saved repository in google drive you can skip cloning anad use the Import Via Google Drive Script)**

In [ ]:
#@title Clone repository from github & allow user to upload dataset zip file (use this if you are running it for the first time)

REPO_URL = "https://github.com/Hugo-Zh0/YoloV12-Object-Detection-Project"
REPO_DIR = "/content/YoloV12-Object-Detection-Project"

import shutil, os
from google.colab import files

# Remove existing repo directory if it exists
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

# Clone the repo
!git clone -q {REPO_URL} {REPO_DIR}
print("Cloned into:", REPO_DIR)

# Prompt user to upload dataset ZIP file
print("lease upload your dataset ZIP file. It will be saved to /content")
uploaded = files.upload()

# Save uploaded ZIP file to /content
for filename in uploaded.keys():
    dest_path = os.path.join('/content', filename)
    print(f"Uploaded file saved as: {dest_path}")

In [ ]:
#@title Import via Google Drive (import your repository) or (dataset) if you have not already trained before

from google.colab import drive
import shutil
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define source and destination paths
source_path = '/content/drive/MyDrive/YoloV12-Object-Detection-Project.zip'  # Update as needed
destination_path = '/content'

# Step 3: Copy from Drive to Colab root
if os.path.isdir(source_path):
    shutil.copytree(source_path, destination_path)
    print(f"Folder copied to {destination_path}")
elif os.path.isfile(source_path):
    shutil.copy2(source_path, destination_path)
    print(f"File copied to {destination_path}")
else:
    print("Source path not found. Please check the path and try again.")

# Step 4: Unmount Google Drive and clean up
drive.flush_and_unmount()
shutil.rmtree('/content/drive', ignore_errors=True)
print("Google Drive unmounted and cleaned up.")

In [8]:
#@title Unzip Respository
!unzip -q /content/YoloV12-Object-Detection-Project.zip -d /content/YoloV12-Object-Detection-Project
!unzip -q /content/runs.zip -d /content/runs

# 3.1 - Creating folders

**(Skip this step if you already have imported a previous repository from google drive)**

Datasets Folder: Contains our datasets from roboflow


Yaml Folder: Contains our data.yaml file from our dataset


In [ ]:
#@title Creates two folders called "datasets & yaml" in the repository
import os
target_directory = '/content/YoloV12-Object-Detection-Project'

folders_to_create = ['datasets', 'yaml']

for folder_name in folders_to_create:
    folder_path = os.path.join(target_directory, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    print(f"Created folder: {folder_path}")

# 3.2 Unzipping Datasets

You should already have your datasets uploaded into the google colab. Via cloning the repo with uploading steps or through Google Drive.

Next we will need to run the scripts below to unzip and move them into the correct directory

In [ ]:
#@title Unzip Koala Dataset
!unzip -q /content/koala.zip -d /content/YoloV12-Object-Detection-Project/datasets

In [ ]:
#@title Unzip Kangaroo Dataset
!unzip -q /content/kangaroo.zip -d /content/YoloV12-Object-Detection-Project/datasets

## Step 4 — 🎞️ Set model & data paths

**Step 4.1**

Manually move YAML File stored in extracted dataset folder to previously created YAML Folder.

In [ ]:
import shutil
import os

# Comment out the path you are not using
source_path = '/content/YoloV12-Object-Detection-Project/datasets/koala'
source_path = '/content/YoloV12-Object-Detection-Project/datasets/kangaroo'
destination_dir = '/content/YoloV12-Object-Detection-Project/yaml'


os.makedirs(destination_dir, exist_ok=True)

destination_path = os.path.join(destination_dir, os.path.basename(source_path))
shutil.move(source_path, destination_path)

print(f"File moved to: {destination_path}")

**Step 4.2**

Update the YAML File with proper location paths for: train, val, test

Double click the yaml file and it will open on the side.

**Koala:**

* /content/YoloV12-Object-Detection-Project/datasets/koala/train/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/test/images


**Kangaroo:**

* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/train/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/test/images

<br>

Finally save the file doing Ctrl+S

## Step 5 — 💥 Train (set your parameters)

This script will train your dataset and store them in the respoitory locations.
You can also set the configs you want to train your dataset, tune it however you like to get the best performance.

In [ ]:
#@title Set Configs and train datasaet

from ultralytics import YOLO
import torch, os, sys, requests

# Change the model dir depending on the training for example:
# On the first train it will create a train folder.
# Then on the second run you change the MODELS-DIR Path to train2/weights to use
# the previous trained best.pt as that is the model

MODELS_DIR  = "/content/YoloV12-Object-Detection-Project/runs/completed-training"
MODEL_PATH  = os.path.join(MODELS_DIR, "yolo12s.pt")
DATA_YAML   = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"
RUNS_DIR    = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

print("Model path:", MODEL_PATH)
print("Data yaml :", DATA_YAML)

# Device selection
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load your local weights explicitly
model = YOLO(MODEL_PATH)

# Training configs
results = model.train(
    data=DATA_YAML, # refers to our dataset yaml file
    epochs=350, #start at 150, 250, 350 (no need to go higher if results becomes same)
    imgsz=960,  #start at 640, 960, 1024 (no need to go higher if results becomes same)
    batch=-1,   #-1 is auto batch (leave yolo to decide how many batches to create based on system)
    workers=2,  #2 is optimal for google colab using T4 GPU, ,experiment to see which is best (before it gives errors)
    device=device, #uses CUDA
    pretrained=False, #prevents using COCO (as its a custom dataset)
    amp=False,        #set to false for automatic mapping matrix (can turn on, experiment however you like)
    project=RUNS_DIR, #saves to custom runs directory MODEL_DIR
    name="train"      #names the folders train
)

print("Save dir:", results.save_dir)

## Step 6 — ✅ Test latest trained model

This script will grab the latest trained runs, uses its best.pt to validate its correct file and do a test on the model to see if detection is happening correctly, looking for accuracy, bounding boxes and efficiency

In [ ]:
#@title Test latest trained model

import glob
import os
from ultralytics import YOLO

RUNS_DIR = "/content/YoloV12-Object-Detection-Project/runs/completed-training"
DATA_YAML = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"

RESULTS_DIR = "/content/YoloV12-Object-Detection-Project/runs/test-results"
os.makedirs(RESULTS_DIR, exist_ok=True)

MODEL_TEST_DIR = "/content/YoloV12-Object-Detection-Project/runs/test-models"
os.makedirs(MODEL_TEST_DIR, exist_ok=True)

# Find the most recent training run
latest_run = max(glob.glob(os.path.join(RUNS_DIR, "*")), key=os.path.getmtime)
weights_path = os.path.join(latest_run, "weights", "best.pt")

print(f"Validating latest run: {latest_run}")
print(f"Using weights: {weights_path}")

# Load the best model from that run
model = YOLO(weights_path)

# Run validation on the TEST split
metrics = model.val(
    data=DATA_YAML,
    split="test",
    imgsz=640,   # you can test 640, 960, 1024 (until you see no further improvement)
    batch=8,     # number of images 8 = 8 images inside yolo automatic batch
    project=MODEL_TEST_DIR,
    name=os.path.basename(latest_run),
    exist_ok=True
)

# Build results filename based on run name (e.g., train2 -> train2_metrics.txt)
run_name = os.path.basename(latest_run)
save_path = os.path.join(RESULTS_DIR, f"{run_name}_metrics.txt")

# Save metrics to file (use results_dict property)
with open(save_path, "w") as f:
    f.write(f"Validation results for run: {run_name}\n")
    f.write(f"Weights: {weights_path}\n\n")
    for k, v in metrics.results_dict.items():
        f.write(f"{k}: {v}\n")

print(f"Metrics saved to {save_path}")
print(f"Ultralytics validation outputs saved to {os.path.join(MODEL_TEST_DIR, run_name)}")


## Step 7 — 🔮 Inference Testing For Images And Videos


After completeing a model test from step 6, we will move onto testing with new images and videos. Which can be sourced online anywhere. This way testing will be more accurate as its new data that the model hasn't been trained/tested on.

In [ ]:
#@title Create folder to store test images and videos **(skip this step if you already have these created previously from running the script)**

import os
from google.colab import files
import shutil

base_dir = "/content/YoloV12-Object-Detection-Project/testing"

# Uncomment the one you are not using
folder_name = "koala"
folder_name = "kangaroo"

target_path = os.path.join(base_dir, folder_name)

os.makedirs(target_path, exist_ok=True)
print(f"Folder created at: {target_path}")

# Prompt user to upload files
print("Please upload your image or video files:")
uploaded = files.upload()

# Move uploaded files into the target folder
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(target_path, filename))
    print(f"Moved '{filename}' to '{target_path}'.")

print("All files uploaded and saved in your custom directory!")

In [ ]:
#@title Inference Testing with Images & Videos
import glob
import os
from ultralytics import YOLO

# Path where YOLO saves runs
RUNS_DIR = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

# Uncomment the one you are not using
TEST_DIR = "/content/YoloV12-Object-Detection-Project/testing/koala"
TEST_DIR = "/content/YoloV12-Object-Detection-Project/testing/kangaroo"

# Base path for inference results
OUTPUT_BASE = os.path.join(TEST_DIR, "inference_results")
os.makedirs(OUTPUT_BASE, exist_ok=True)

# Find the next available inference folder (inference1, inference2, ...)
i = 1
while os.path.exists(os.path.join(OUTPUT_BASE, f"inference{i}")):
    i += 1
OUTPUT_DIR = os.path.join(OUTPUT_BASE, f"inference{i}")

# Find the most recent training run
latest_run = max(glob.glob(os.path.join(RUNS_DIR, "*")), key=os.path.getmtime)
weights_path = os.path.join(latest_run, "weights", "best.pt")

print(f"Running inference with weights: {weights_path}")
print(f"Testing folder: {TEST_DIR}")
print(f"Results will be saved to: {OUTPUT_DIR}")

# Load model
model = YOLO(weights_path)

# Run inference (images + videos in same folder)
results = model.predict(
    source=TEST_DIR,   # folder containing both images & videos
    imgsz=960,         # change between 540 960 1024 or more
    conf=0.70,         # confidence higher means only detect if 0.70 or above (experiment to find the best conf)
    save=True,
    project=OUTPUT_DIR,
    name="",           # ensures results are saved directly in OUTPUT_DIR
    exist_ok=True
)

print(f"Inference complete. Results saved to: {OUTPUT_DIR}")



# Step 8 - 🪣 Choosing Final Model for Deployment

This step will go through every single trained model/validation results, and finds/selects the best model from the metrics provided. This way with the Final Model selected will go through final inference testing then Step 9 to be exported and used in real world deployment.

In [ ]:
#@title Find the best final model to use from test results (saves as a csv)

import glob
import os
import pandas as pd

# Paths
TEST_RESULTS_DIR = "/content/YoloV12-Object-Detection-Project/runs/test-results"
RESULTS_DIR = "/content/YoloV12-Object-Detection-Project/runs/model-results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Output CSV path
CSV_PATH = os.path.join(RESULTS_DIR, "model_comparison.csv")

rows = []

for path in sorted(glob.glob(os.path.join(TEST_RESULTS_DIR, "*_metrics.txt"))):
    with open(path, "r") as f:
        lines = [ln.strip() for ln in f.readlines()]

    # Defaults
    run = ""
    weights = ""
    precision = recall = map50 = map5095 = fitness = None

    for ln in lines:
        if ln.startswith("Validation results for run:"):
            run = ln.split("Validation results for run:", 1)[1].strip()
        elif ln.startswith("Weights:"):
            weights = ln.split("Weights:", 1)[1].strip()
        elif ln.startswith("metrics/precision(B):"):
            precision = float(ln.split(":", 1)[1].strip())
        elif ln.startswith("metrics/recall(B):"):
            recall = float(ln.split(":", 1)[1].strip())
        elif ln.startswith("metrics/mAP50(B):"):
            map50 = float(ln.split(":", 1)[1].strip())
        elif ln.startswith("metrics/mAP50-95(B):"):
            map5095 = float(ln.split(":", 1)[1].strip())
        elif ln.startswith("fitness:"):
            fitness = float(ln.split(":", 1)[1].strip())

    if run and weights:
        rows.append({
            "run": run,
            "weights": weights,
            "mAP50": map50,
            "mAP50-95": map5095,
            "precision": precision,
            "recall": recall,
            "fitness": fitness,
            "metrics_file": os.path.basename(path),
        })

# Save results to CSV
df = pd.DataFrame(rows)
df.to_csv(CSV_PATH, index=False)

print(f"\nModel comparison saved to {CSV_PATH}")
print(df)




In [ ]:
#@title Compares Model Metrics in CSV and chooses the best model

import os
import shutil
import pandas as pd

#@title Compares Model Metrics in CSV and chooses the best model

import os
import shutil
import pandas as pd

# ==== CONFIG ====
CSV_PATH = "/content/YoloV12-Object-Detection-Project/runs/model-results/model_comparison.csv"
MODELS_DIR = "/content/YoloV12-Object-Detection-Project/models"
SUMMARY_DIR = "/content/YoloV12-Object-Detection-Project/runs/model-results"  # fixed path

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

# loads csv file
df = pd.read_csv(CSV_PATH)

# checks csv columns (correct data)
required_cols = {"run", "weights", "mAP50-95", "mAP50", "precision", "recall"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"CSV missing required columns: {missing}")

for c in ["mAP50-95", "mAP50", "precision", "recall"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# picks best model from csv data metrics
best_row = df.sort_values(
    by=["mAP50-95", "mAP50", "precision"],  # tie-break if needed
    ascending=[False, False, False]
).iloc[0]

run = best_row["run"]
src = best_row["weights"]

if not isinstance(src, str) or not os.path.exists(src):
    raise FileNotFoundError(f"best.pt not found: {src}")

# Destination name: trainxxx_best.pt
base_name = f"{run}_best.pt"
dst = os.path.join(MODELS_DIR, base_name)

print(f"   Best model found: {run}")
print(f"   mAP50-95: {best_row['mAP50-95']:.4f} | mAP50: {best_row['mAP50']:.4f} | precision: {best_row['precision']:.4f}")
print(f"   Copying: {src} -> {dst}")

shutil.copy2(src, dst)

# Output and saving summary selection
summary_csv = os.path.join(SUMMARY_DIR, "best_model_summary.csv")
best_row.to_frame().T.to_csv(summary_csv, index=False)
print(f"\n Summary written to: {summary_csv}")




## Step 9 — ♟️ Exporting

Export the full respository incase you run into errors, you think the runtime is about to expire or you just want to save it to run later on.

In [ ]:
#@title Export Repository as Zip File
import shutil

shutil.make_archive('YoloV12-Object-Detection-Project', 'zip', '/content/YoloV12-Object-Detection-Project')

from google.colab import files
files.download('YoloV12-Object-Detection-Project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## This will become a script to export as a ready model file

## Troubleshooting


- **Weights YAML missing:** ensure `models/yolo12s.pt` and `yaml/data.yaml` exist in the repo or update paths.
- **Val fails:** Train first; then rerun the validate cell.
- **Poor metrics:** Add more data, correct labels, tune `imgsz`/`batch`/`epochs`.
